In [ ]:
!pip install transformers transformers[torch] datasets

Fine Tunning

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
model_name = "pierreguillou/gpt2-small-portuguese"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
# dados para fine tunning
import requests
from datasets import Dataset

url = "https://pt.wikipedia.org/wiki/Ci%C3%AAncia"
response = requests.get(url)
text_data = response.text

lines = text_data.split('\n')
data_dict = {"text": lines}

dataset = Dataset.from_dict(data_dict)

In [ ]:
# tokenização do dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1888 [00:00<?, ? examples/s]

In [ ]:
first_example = tokenized_dataset[1250]
print(first_example)

{'text': '<p>Os termos "<a href="/wiki/Modelo_cient%C3%ADfico" title="Modelo científico">modelo</a>", "<a href="/wiki/Hip%C3%B3tese" title="Hipótese">hipótese</a>", "<a href="/wiki/Lei_f%C3%ADsica" class="mw-redirect" title="Lei física">lei</a>" e "<a href="/wiki/Teoria" title="Teoria">teoria</a>" têm significados diferentes em ciência e na linguagem coloquial.<span style="color:gray"><sup>&#91;</sup></span><sup><a href="/wiki/Wikip%C3%A9dia:Livro_de_estilo/Cite_as_fontes" title="Wikipédia:Livro de estilo/Cite as fontes"><span title="Esta afirmação precisa de uma referência para confirmá-la desde maio de 2016." style="color:gray"><i>carece&#160;de fontes</i></span></a><span class="printfooter">?</span><span style="color:gray">&#93;</span></sup>', 'input_ids': [28, 80, 30, 1220, 3387, 330, 28, 65, 469, 3281, 7380, 15, 87, 4628, 15, 28761, 934, 63, 19540, 5, 35, 19, 5, 11665, 16171, 2, 4344, 320, 7380, 28761, 934, 8936, 12624, 28334, 5667, 65, 30, 474, 330, 28, 65, 469, 3281, 7380, 15, 8

In [ ]:
data_collator = DataCollatorForLanguageModeling( tokenizer=tokenizer,mlm=False)

In [ ]:
# hiper parâmetros
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1, # !!!
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2
)

In [ ]:
trainer = Trainer(model=model,args=training_args, data_collator=data_collator, train_dataset=tokenized_dataset)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=59, training_loss=2.856429795087394, metrics={'train_runtime': 60.5256, 'train_samples_per_second': 31.193, 'train_steps_per_second': 0.975, 'total_flos': 123329839104000.0, 'train_loss': 2.856429795087394, 'epoch': 1.0})

In [ ]:
# Salvar
model.save_pretrained("meu_modelo_treinado")
tokenizer.save_pretrained("meu_modelo_treinado")

('meu_modelo_treinado/tokenizer_config.json',
 'meu_modelo_treinado/special_tokens_map.json',
 'meu_modelo_treinado/vocab.json',
 'meu_modelo_treinado/merges.txt',
 'meu_modelo_treinado/added_tokens.json')

Usando o Modelo

In [ ]:
from transformers import pipeline

In [ ]:
gerador = pipeline("text-generation", model="meu_modelo_treinado")
texto = "Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico."
resultado = gerador(texto, max_length=60, do_sample=True, temperature=0.7, top_k=50, top_p=0.85)
print(resultado)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico. O conhecimento científico é uma ciência que estuda os fenômenos naturais, e a ciência aplicada, que estuda os fenômenos naturais, é uma ciência que estuda os fenômenos naturais.\n\nO estudo científico é o estudo da ciência'}]
